In [1]:
!pip install gradio opencv-python-headless numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [3]:
import cv2
import numpy as np
import gradio as gr

In [4]:
# Contrast Enhancement
def increase_contrast(img, factor=5):
    img_float = img.astype(np.float32) * factor  # Multiply pixel values
    img_float = np.clip(img_float, 0, 255)  # Clip to valid range
    return img_float.astype(np.uint8)

In [5]:
# Gamma Correction
def gamma_correction(img, gamma):
    table = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

In [6]:
# CLAHE (Adaptive Histogram Equalization)
def clahe_enhancement(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # Convert to LAB color space
    l, a, b = cv2.split(lab)  # Split channels
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))  # CLAHE
    l = clahe.apply(l)  # Apply CLAHE to L-channel
    return cv2.cvtColor(cv2.merge((l, a, b)), cv2.COLOR_LAB2BGR)  # Convert back to BGR

In [7]:
# Retinex (Single-Scale Retinex)
def retinex(img, sigma=50):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    log_img = np.log1p(np.float32(gray))
    blur = cv2.GaussianBlur(log_img, (0, 0), sigma)
    retinex_img = log_img - blur
    retinex_img = np.uint8(np.clip((retinex_img - retinex_img.min()) * 255 / (retinex_img.max() - retinex_img.min()), 0, 255))
    return cv2.cvtColor(retinex_img, cv2.COLOR_GRAY2BGR)

In [8]:
# Noise Reduction (Bilateral Filtering)
def denoise_image(img):
    return cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

In [9]:
# Final Enhancement Function
def enhance_image(image, gamma=0.5, apply_clahe=True, apply_retinex=True, apply_denoise=True):
    contrast_img = increase_contrast(image)  # Increase contrast
    gamma_img = gamma_correction(image, gamma)  # Gamma correction

    if apply_clahe:
        contrast_img = clahe_enhancement(contrast_img)  # Apply CLAHE

    if apply_retinex:
        retinex_img = retinex(image)  # Apply Retinex
        contrast_img = cv2.addWeighted(contrast_img, 0.7, retinex_img, 0.3, 0)  # Merge with Retinex

    if apply_denoise:
        contrast_img = denoise_image(contrast_img)  # Apply Noise Reduction

    enhanced_img = cv2.addWeighted(gamma_img, 0.6, contrast_img, 0.4, 0)  # Weighted Fusion
    return enhanced_img

In [10]:
# Gradio UI Function
def process_image(input_img, gamma, apply_clahe, apply_retinex, apply_denoise):
    return enhance_image(input_img, gamma, apply_clahe, apply_retinex, apply_denoise)

# Gradio Interface
interface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="numpy"),  # Input image
        gr.Slider(0.1, 3.0, value=0.5, label="Gamma Correction"),
        gr.Checkbox(label="Apply CLAHE (Adaptive Histogram Equalization)", value=True),
        gr.Checkbox(label="Apply Retinex (Enhance Dark Regions)", value=True),
        gr.Checkbox(label="Apply Denoising (Reduce Noise)", value=True),
    ],
    outputs=gr.Image(type="numpy"),  # Output image
    title="🌙 Advanced Low-Light Image Enhancement ☀️",
    description="Upload a low-light image and enhance brightness & contrast using various techniques.",
)

# Launch Gradio App
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3905467f297661f2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
